# Frequent Pattern (FP) Growth Algorithm

* `FP-growth algorithm` is an improved version of the Apriori algorithm used for Association Rule Mining (aka. Frequent Pattern mining) from the database. 

* `FP-growth algorithm` is a tree-based algorithm for frequent itemset mining or frequent-pattern mining used for market basket analysis.
* The algorithm represents the data in a tree structure known as `FP-tree` (frequent pattern tree).  
    * `FP-tree` is responsible for maintaining the association information between the frequent items.

* FP Growth is one of the associative rule learning techniques which is used in machine learning for finding frequently occurring patterns.

* Several data items are connected together based on certain rules, which discovers the relations between them in large databases.

The Apriori algorithms have two significant drawbacks: 
* speed
* high computational cost
   

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = 'https://raw.githubusercontent.com/tec03/Datasets/main/datasets/fpEg1.csv'

In [3]:
df = pd.read_csv(path, 
                 header = None)
df

,0,1,2,3,4,5,6,7
0,a,c,d,f,g,i,m,p
1,a,b,c,f,l,m,o,NaN
2,b,f,h,j,o,NaN,NaN,NaN
3,b,c,k,s,p,NaN,NaN,NaN
4,a,c,e,f,i,p,m,n


## Tree graph

Gather All Items of Each Transactions into Numpy Array

In [4]:
transaction = []
for i in range(0, df.shape[0]):
    for j in range(0, df.shape[1]):
        transaction.append(df.values[i,j])

transaction

['a',
 'c',
 'd',
 'f',
 'g',
 'i',
 'm',
 'p',
 'a',
 'b',
 'c',
 'f',
 'l',
 'm',
 'o',
 nan,
 'b',
 'f',
 'h',
 'j',
 'o',
 nan,
 nan,
 nan,
 'b',
 'c',
 'k',
 's',
 'p',
 nan,
 nan,
 nan,
 'a',
 'c',
 'e',
 'f',
 'i',
 'p',
 'm',
 'n']

Convert to numpy array

In [5]:
transaction = np.array(transaction)
transaction

array(['a', 'c', 'd', 'f', 'g', 'i', 'm', 'p', 'a', 'b', 'c', 'f', 'l',
       'm', 'o', 'nan', 'b', 'f', 'h', 'j', 'o', 'nan', 'nan', 'nan', 'b',
       'c', 'k', 's', 'p', 'nan', 'nan', 'nan', 'a', 'c', 'e', 'f', 'i',
       'p', 'm', 'n'], dtype='<U32')

Transform Them a Pandas DataFrame

In [6]:
ndf = pd.DataFrame(transaction, 
                  columns=["items"]
                 ) 
ndf.head()

,items
0,a
1,c
2,d
3,f
4,g


Put 1 to Each Item For Making Countable Table, to be able to perform Group By:

In [7]:
ndf["incident_count"] = 1 
ndf.head()

,items,incident_count
0,a,1
1,c,1
2,d,1
3,f,1
4,g,1


In [8]:
ndf[ndf['items'] == "nan"]

,items,incident_count
15,nan,1
21,nan,1
22,nan,1
23,nan,1
29,nan,1
30,nan,1
31,nan,1


Delete `NaN` Items from the row:

In [9]:
id_to_delete = ndf[ndf['items'] == "nan" ].index
id_to_delete

Int64Index([15, 21, 22, 23, 29, 30, 31], dtype='int64')

In [10]:
ndf.drop(id_to_delete , 
         inplace=True
        )
ndf[ndf['items'] == "nan"]

,items,incident_count


Make an Appropriate Pandas DataFrame for Explanotory analisis: 

In [11]:
df_table = ndf.groupby("items").sum().sort_values("incident_count", 
                                                 ascending=False
                                                ).reset_index()
df_table.head()

,items,incident_count
0,c,4
1,f,4
2,a,3
3,b,3
4,p,3


Initial Visualization:

In [12]:
df_table.head().style.background_gradient(cmap='Blues')

,items,incident_count
0,c,4
1,f,4
2,a,3
3,b,3
4,p,3


In [13]:
df_table["all"] = "with desired threshold" # to have a same origin
df_table.head()

,items,incident_count,all
0,c,4,with desired threshold
1,f,4,with desired threshold
2,a,3,with desired threshold
3,b,3,with desired threshold
4,p,3,with desired threshold


Create tree map:

In [14]:
import plotly.express as px

fig = px.treemap(df_table.head(50),
                 path=['all', "items"], 
                 values='incident_count',
                 color=df_table["incident_count"].head(50), 
                 hover_data=['items'],
                 color_continuous_scale='Blues',
                )
fig.show();

/usr/local/lib/python3.9/dist-packages/plotly/express/_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
/usr/local/lib/python3.9/dist-packages/plotly/express/_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)


## Data pre-processing

Transform every transaction to seperate list and gather them into Numpy array: 

In [15]:
transaction = []
for i in range(df.shape[0]):
    transaction.append([str(df.values[i,j]) 
                        for j in range(df.shape[1])
                       ])

transaction = np.array(transaction)# creating the numpy array of the transactions
transaction

array([['a', 'c', 'd', 'f', 'g', 'i', 'm', 'p'],
       ['a', 'b', 'c', 'f', 'l', 'm', 'o', 'nan'],
       ['b', 'f', 'h', 'j', 'o', 'nan', 'nan', 'nan'],
       ['b', 'c', 'k', 's', 'p', 'nan', 'nan', 'nan'],
       ['a', 'c', 'e', 'f', 'i', 'p', 'm', 'n']], dtype='<U3')

In [16]:
# Alternatively

ndf = df.applymap(str)
trans = ndf.values.tolist()
type(trans[1][7])

str

In [17]:
print(type(transaction))
print(type(trans))

<class 'numpy.ndarray'>
<class 'list'>


In [18]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)

encoded_df = pd.DataFrame(te_ary, 
                          columns=te.columns_
                         )
encoded_df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,nan,o,p,s
0,True,False,True,True,False,True,True,False,True,False,False,False,True,False,False,False,True,False
1,True,True,True,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False
2,False,True,False,False,False,True,False,True,False,True,False,False,False,False,True,True,False,False
3,False,True,True,False,False,False,False,False,False,False,True,False,False,False,True,False,True,True
4,True,False,True,False,True,True,False,False,True,False,False,False,True,True,False,False,True,False


Select top 10 items, just to test purposes. Then, we repeat the same with entire data.

In [19]:
first10 = encoded_df.head(10)#.values 
first10
##dataset = first10.loc[:,first10] # Extract Top 10
dataset = encoded_df
dataset.shape

(5, 18)

In [20]:
print(type(encoded_df))
encoded_df.columns

<class 'pandas.core.frame.DataFrame'>


Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'nan', 'o', 'p', 's'],
      dtype='object')

In [21]:
encoded_df.drop(['nan'],
                axis = 1,
                inplace = True
               )
encoded_df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,s
0,True,False,True,True,False,True,True,False,True,False,False,False,True,False,False,True,False
1,True,True,True,False,False,True,False,False,False,False,False,True,True,False,True,False,False
2,False,True,False,False,False,True,False,True,False,True,False,False,False,False,True,False,False
3,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,True,True
4,True,False,True,False,True,True,False,False,True,False,False,False,True,True,False,True,False


### FP growth algorithm

In [ ]:
pip uninstall mlxtend

In [23]:
pip install mlxtend

In [24]:
from mlxtend.frequent_patterns import fpgrowth


frequent_itemsets = fpgrowth(encoded_df,
                             min_support=0.6, 
                             use_colnames=True
                            )
frequent_itemsets#.head(11)

,support,itemsets
0,0.8,(f)
1,0.8,(c)
2,0.6,(p)
3,0.6,(m)
4,0.6,(a)
5,0.6,(b)
6,0.6,"(f, c)"
7,0.6,"(p, c)"
8,0.6,"(m, c)"
9,0.6,"(m, f)"


### Association rules

In [25]:
from mlxtend.frequent_patterns import association_rules

as_ruls = association_rules(frequent_itemsets, 
                                      metric = "lift", 
                                      min_threshold = 1.5
                                     )


as_ruls

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(f, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf
1,(m),"(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
2,(m),(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf
3,(a),(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf
4,"(m, c)",(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf
5,"(a, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf
6,(m),"(a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
7,(a),"(m, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
8,"(m, f)",(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf
9,"(f, a)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf


Now, let's sort values based on confidence:

In [26]:
sortedRlz = as_ruls.sort_values("confidence",
                               ascending=False).head(10)
sortedRlz

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(f, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf
1,(m),"(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
22,(m),"(f, a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
21,"(a, c)","(m, f)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
20,"(f, c)","(m, a)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
19,"(f, a)","(m, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
18,"(m, c)","(f, a)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
17,"(m, a)","(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
16,"(m, f)","(a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
15,"(f, a, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf


### Conclusions:

 

\begin{array}{lcl}
\text{\{c, f, m\}}     & :&  \text{support = 0.6, means out of 5 transactions 3 has \{m,c\}}.  \\
\text{\{m\}}     & :&  \text{antecendent and consequent support means 0.6 (3/5) and 0.6 (3/5)}\\
                    && \text{Same can be observed in product_count table }\\
\text{\{m, c, f\}}     & :&  \text{support = 0.6, means out of 5 transactions 3 has {m,a,f}}. \\
\text{\{f, a, m, c\}}     & :&  \text{support = 0.6, means out of 5 transactions 3 has {f,a,m,c}}. 
\end{array}


### Case1: We want `antecedentes` only `m`: 

In [27]:
sortedRlz['antecedents']

0        (f, c)
1           (m)
22          (m)
21       (a, c)
20       (f, c)
19       (f, a)
18       (m, c)
17       (m, a)
16       (m, f)
15    (f, a, c)
Name: antecedents, dtype: object

In [28]:
sortedRlz[
    sortedRlz['antecedents'] == {'m'}#frozenset(('a'))
]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(m),"(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
22,(m),"(f, a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf


### Case2: We want to remove  `antecedents (c, f) consequents (m)`:

In [29]:
frozenset({'f', 'c'}) #Frozen set is just an immutable version of a Python set object.

frozenset({'c', 'f'})

In [30]:
ante = sortedRlz['antecedents'] == frozenset({'f', 'c'})  #if antecedents are (f, c), it will be True. False otherwise

In [31]:
anteDF = pd.concat([sortedRlz['antecedents'], 
                    ante
                   ], 
                   axis = 1
                  )
anteDF

,antecedents,antecedents
0,"(f, c)",True
1,(m),False
22,(m),False
21,"(a, c)",False
20,"(f, c)",True
19,"(f, a)",False
18,"(m, c)",False
17,"(m, a)",False
16,"(m, f)",False
15,"(f, a, c)",False


In [32]:
cons = sortedRlz['consequents'] == frozenset({'m'})

In [33]:
consDF = pd.concat([sortedRlz['consequents'], 
                    cons
                   ], 
                   axis = 1
                  )
consDF

,consequents,consequents
0,(m),True
1,"(f, c)",False
22,"(f, a, c)",False
21,"(m, f)",False
20,"(m, a)",False
19,"(m, c)",False
18,"(f, a)",False
17,"(f, c)",False
16,"(a, c)",False
15,(m),True


In [34]:
Nos = (ante & cons)
NosDF = pd.concat([ante,
                  cons,
                  Nos,
                   sortedRlz.antecedents, 
                   sortedRlz.consequents
                   ], 
                   axis = 1
                  )
NosDF

,antecedents,consequents,0,antecedents,consequents
0,True,True,True,"(f, c)",(m)
1,False,False,False,(m),"(f, c)"
22,False,False,False,(m),"(f, a, c)"
21,False,False,False,"(a, c)","(m, f)"
20,True,False,False,"(f, c)","(m, a)"
19,False,False,False,"(f, a)","(m, c)"
18,False,False,False,"(m, c)","(f, a)"
17,False,False,False,"(m, a)","(f, c)"
16,False,False,False,"(m, f)","(a, c)"
15,False,True,False,"(f, a, c)",(m)


In [35]:
sortedRlz.loc[ Nos ]#will return the where  antecedents are (c,f) and consequents(m). 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(f, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf


In [36]:
sortedRlz.loc[ ~Nos ]#will return the where  antecedents are not (c,f) and consequents(m). 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(m),"(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
22,(m),"(f, a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
21,"(a, c)","(m, f)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
20,"(f, c)","(m, a)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
19,"(f, a)","(m, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
18,"(m, c)","(f, a)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
17,"(m, a)","(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
16,"(m, f)","(a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf
15,"(f, a, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf


In short : 

In [37]:
ante = sortedRlz['antecedents'] == frozenset({'f', 'c'}) 
cons = sortedRlz['consequents'] == frozenset({'m'})
Nos = (ante & cons)

sortedRlz.loc[Nos] #~Nos for the compliment

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(f, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf


### Case3:  We are ony interested in rules that satisfy the following criteria:

1. at least 3 antecedents
2. a confidence > 0.8
3. a lift score > 1.5

For, let's create a new column `ante_len` to have the lenght of `antecedents`: 

In [38]:
as_ruls["ante_len"] = as_ruls["antecedents"].apply(lambda x: len(x))
as_ruls

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,ante_len
0,"(f, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2
1,(m),"(f, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf,1
2,(m),(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1
3,(a),(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf,1
4,"(m, c)",(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2
5,"(a, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2
6,(m),"(a, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf,1
7,(a),"(m, c)",0.6,0.6,0.6,1.0,1.666667,0.24,inf,1
8,"(m, f)",(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2
9,"(f, a)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2


In [39]:
as_ruls[ 
    (as_ruls['ante_len'] >= 3) &
    (as_ruls['confidence'] > 0.8) &
    (as_ruls['lift'] > 1.5) 
]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,ante_len
14,"(m, f, c)",(a),0.6,0.6,0.6,1.0,1.666667,0.24,inf,3
15,"(f, a, c)",(m),0.6,0.6,0.6,1.0,1.666667,0.24,inf,3


<!--NAVIGATION-->
< [previous](https://github.com/Egade/ClassNotes) | [Contents](https://github.com/Egade/ClassNotes) | [next](https://github.com/Egade/ClassNotes) >